In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./极米科技投资者问答.csv')
data=data.loc[:, ~data.columns.str.contains('^Unnamed')]

In [6]:
data.head()

,提问时间,问题,回答,回答时间,来源
0,20210927 09:03,:极米科技(688696)请问董秘，公司受本次全国性限电影响多大？如何应对？,尊敬的投资者，您好，感谢您的关注，目前公司经营生产活动一切正常，暂未收到限电通知。,09月30日 18:10,网站
1,20210916 14:17,:极米科技(688696)你们基本面出了啥问题么？怎么股价一直跌？还是因为芯片供应出了大问题？,尊敬的投资者，您好！今年以来，受到疫情、供需等多重因素影响，公司产品使用的DMD芯片等电子料...,09月22日 15:53,网站
2,20210913 14:45,:极米科技(688696)请问贵公司对激光微投是否有产品布局？如没有，是出于何种考量？,尊敬的投资者，您好！LED微投与激光微投主要差异在于光源是选择LED还是激光。激光光源的一个...,09月15日 17:25,网站
3,20210830 14:23,:极米科技(688696)贵公司投影仪能通过vr眼镜，观看3d效果电影，或者玩3d游戏么？,尊敬的投资者，您好！感谢您对公司的关注，公司产品支持3D影片的播放，选配极米3D眼镜观看即可...,08月31日 15:49,网站
4,20210804 08:49,:极米科技(688696)公司最新股东人数,尊敬的投资者，您好！感谢您对公司的关注，股东人数请查阅公司披露的定期报告，谢谢！,08月10日 17:33,网站


In [4]:
import re

In [5]:
data['提问时间'] = data['提问时间'].apply(lambda x:'2021'+''.join(re.split('[月日]', x)))

In [12]:
date = pd.to_datetime(data['提问时间'])

In [19]:
hour=[i.hour for i in date]
# hour

In [20]:
from collections import Counter

In [32]:
time = list(Counter(hour).keys())
num = list(Counter(hour).values())

In [25]:
from pyecharts.charts import Bar

In [36]:
bar = Bar()
bar.add_xaxis(time)
bar.add_yaxis('',num)
# bar.render_notebook()

In [40]:
data['回答时间'] = data['回答时间'].apply(lambda x:'2021'+''.join(re.split('[月日]', x)))
date_reply = pd.to_datetime(data['回答时间'])

In [53]:
time_reply = list(Counter((date_reply-date).map(lambda x:x.days)).keys())
num_reply = list(Counter((date_reply-date).map(lambda x:x.days)).values())

In [55]:
bar_reply = Bar()
bar_reply.add_xaxis(time_reply)
bar_reply.add_yaxis('',num_reply)
# bar_reply.render_notebook()

In [56]:
import os

In [57]:
filedir = '/Users/Lee/Desktop/hui/stopwords-master/'
filenames=os.listdir(filedir)
f=open('stopwordsAll.txt','w')
#先遍历文件名
for filename in filenames:
    filepath = filedir+'/'+filename
    #遍历单个文件，读取行数
    for line in open(filepath):
        f.writelines(line)
    f.write('\n')
#关闭文件
f.close()

In [58]:
readDir = "./stopwordsAll.txt"
writeDir = "./stopwordsGeneral.txt"
outfile=open(writeDir,"w")
f = open(readDir,"r")
lines_seen = set() 
for line in f:
    line = line.strip('\n')
    if line not in lines_seen:
        outfile.write(line+ '\n')
        lines_seen.add(line)

In [61]:
question=data['问题'].apply(lambda x:x.split(':极米科技(688696)')).apply(lambda x:x[1])
# question

In [63]:
from paddlenlp import Taskflow
seg = Taskflow("word_segmentation")

In [67]:
content = ''.join(list(question))

In [68]:
seg_list = seg(content)

In [72]:
stopwords = [line.strip() for line in open('./stopwordsGeneral.txt', 'r').readlines()]

In [79]:
outstr = '' # 待返回字符串
for word in list(seg_list):
    if word not in stopwords:
        outstr += word + " "

In [83]:
# outstr

In [81]:
def clearSen(comment):
    comment = comment.strip()
    comment = comment.replace('、', '')
    comment = comment.replace('，', '。')
    comment = comment.replace('《', '。')
    comment = comment.replace('》', '。')
    comment = comment.replace('～', '')
    comment = comment.replace('…', '')
    comment = comment.replace('\r', '')
    comment = comment.replace('\t', ' ')
    comment = comment.replace('\f', ' ')
    comment = comment.replace('/', '')
    comment = comment.replace('、', ' ')
    comment = comment.replace('/', '')
    comment = comment.replace('。', '')
    comment = comment.replace('（', '')
    comment = comment.replace('）', '')
    comment = comment.replace('_', '')
    comment = comment.replace('?', ' ')
    comment = comment.replace('？', ' ')
    comment = comment.replace('了', '')
    comment = comment.replace('➕', '')
    comment = comment.replace('：', '')
    return comment

In [86]:
outstr_cS = clearSen(outstr)

In [113]:
ret = re.sub('[公司,请问，董秘，贵，贵司,您好,尊敬]','',outstr_cS)

In [115]:
ret_Split = ret.split(' ')

In [116]:
ret_Strip=[x.strip() for x in ret_Split if x.strip() != '']

In [117]:
counter_ret = Counter(ret_Strip)

In [118]:
from pyecharts.charts import WordCloud

In [119]:
result = [(name,value) for name,value in counter_ret.items()]

In [120]:
wordcloud = WordCloud()

In [121]:
wordcloud.add("", result)
wordcloud.render_notebook()

In [122]:
source = data['来源']

In [126]:
source_Count = [(source,count) for source,count in Counter(source).items()]

In [127]:
from pyecharts.charts import Pie

In [128]:
pie = Pie()
pie.add('',source_Count)
pie.render_notebook()